In [182]:
import numpy as np

# The 0-1 Knapsack Problem (Combinatorial Optimization)

Given container of maximum capacity $c$ and a set of items each with:
*   a weight $w$
*   a value $v$

determine the subset of items that fits in the container such that the value of the set is maximized.

## Formulation

maximize $$\sum_{i=1}^{n}v_i x_i$$

such that $$\sum_{i=1}^{n}w_i x_i \leq c$$

where:

*   $v_i$ is the value of item $i$
*   $w_i$ is the weight of item $i$
*   $x_i \in \{0, 1\}$ is the boolean condition of whether item $i$ is in the bag
*   $c$ is the maximum capacity of the container

## Binary Encoding

Given a mapping of $n$ values $V$, a binary mapping $W$ of $n$ bits can be derived.

In [183]:
# the value mapping represented as a list of values
V = np.array([
    10,
    5,
    1
])

# the binary encoding represented as a list of bits
W = np.array([
    1,
    0,
    1
])

## Evaluation

In [184]:
(V*W).sum()

11

## Problem Domain

In [204]:
mapping = np.random.randint(low=1, high=100, size=100)
mapping

array([54, 49,  4, 90, 30, 82, 42, 26, 69, 73,  8, 44,  8, 53, 50,  2, 81,
       13, 20, 39, 39, 85, 16,  3, 17, 73, 88,  7, 66, 41, 79, 85, 59, 37,
       98, 60,  1, 57, 19, 59, 66, 51, 62, 83, 73, 29, 27, 48, 59, 68,  9,
        6, 81, 69, 34, 89, 92, 28, 13, 72, 94, 30, 31, 11, 30, 22, 65, 73,
       64, 50, 32, 84, 68, 47, 85, 12, 60, 54, 70, 86, 48, 99, 96, 85, 56,
       57, 27, 78, 42, 92,  9, 61, 21, 31, 31, 92,  6, 67, 96, 32])

In [205]:
capacity = np.random.randint(low=1000, high=10000)

In [206]:
capacity

6971

## OOP

In [299]:
V = np.array([
    10.2,
    5,
    1
])

def evaluate_arb(W: np.array):
    return np.sum(W * V)

In [370]:
from typing import Callable

class Chromosome:
    pass

class BinaryChromosome(Chromosome):
    """This class represents a binary encoded chromosome."""
    
    # the valid initial state options for how to initialize the chromosome
    INITIAL_STATES = ['zeros', 'ones', 'random']
    
    def __init__(self, size: int=0, initial_state='random', evaluate: Callable=None):
        """Initialize a new chromosome of a given size.
        
        Args:
            size: the size of the chromosome (default 0) 
            intial_state: the initial state of the chromosome (default 'random')
                * can be 'zeros', 'ones', or 'random'
        """
        # check the validity of the size parameter
        if not isinstance(size, (int, float)):
            raise ValueError('size should be a numeric value like: int, float')
        elif size < 0:
            raise ValueError('cannot create chromosome with a negative size')
        # check if initializing with zeros
        if initial_state == self.INITIAL_STATES[0]:
            self.W = np.zeros(size).astype(int)
        # check if intializing with ones
        elif initial_state == self.INITIAL_STATES[1]:
            self.W = np.ones(size).astype(int)
        # check if initializing with random values
        elif initial_state == self.INITIAL_STATES[2]:
            self.W = np.random.randint(low=0, high=2, size=size).astype(int)
        # invalid initial state, raise error
        else:
            raise ValueError('initial_state must be one of: {}'.format(self.INITIAL_STATES))
        # validate the evaluation function
        if evaluate is None:
            print('WARNING: created a chromosome without an evaluation function')
        elif not isinstance(evaluate, Callable):
            raise ValueError('`evaluate` must be a function')
        self.evaluate = evaluate
  
    def __repr__(self) -> str:
        """Return a string represtation of this object"""
        return str(self.W)
            
    @property
    def fitness(self) -> float:
        # if the evaluation was never instantiated, the score is 0
        if self.evaluate is None:
            return 0
        else:
            return self.evaluate(self.W)
    

In [344]:
o = BinaryChromosome(size=4, initial_state='ones')
o.W

array([1, 1, 1, 1])

In [345]:
r = BinaryChromosome(size=4, initial_state='random')
r.W

array([0, 1, 0, 0])

In [300]:
BinaryChromosome(initial_state=None)

ValueError: initial_state must be one of: ['zeros', 'ones', 'random']

In [283]:
BinaryChromosome(size='asdf')

ValueError: size should be a numeric value like: int, float

In [284]:
BinaryChromosome(size=-1)

ValueError: cannot create chromosome with a negative size

In [346]:
BinaryChromosome(3)

[1 1 0]

In [349]:
c = BinaryChromosome(size=3, evaluate=evaluate_arb)
print(c)
print(c.fitness)

[1 1 0]
15.2


In [364]:
class ChromosomeFactory:
    """A class for generating new chromosomes."""
    
    def __init__(self,
                 chromosome_class: Chromosome, 
                 chromosome_size: int,
                 initial_state: str='random',
                 evaluate: Callable = None):
        """
        Initialize a new chromosome factory.
        
        Args:
            chromosome_class: the class of the chromosomes to create
            chromosome_size: the size of the chromsome
            initial_state: the initial state of the chromosome (default 'random')
            evaluate: the evaluation function for the chromosome (default None)
        """
        if not issubclass(chromosome_class, Chromosome):
            raise ValueError('chromosome_class must be a subclass of Chromosome')
        self.chromosome_class = chromosome_class
        if not isinstance(chromosome_size, (float, int)):
            raise ValueError('chromosome_size must be a numeric like: float, int')
        self.chromosome_size = chromosome_size
        if not isinstance(initial_state, str):
            raise ValueError('initial_state must be a string')
        self.initial_state = initial_state
        if not isinstance(evaluate, Callable):
            raise ValueError('evaluate must be a callable method')
        self.evaluate = evaluate
        
    @property
    def next_individual(self):
        """Create and return a new individual."""
        return self.chromosome_class(size=self.chromosome_size, 
                                     initial_state=self.initial_state, 
                                     evaluate=self.evaluate)
    
    def population(self, size: int):
        """Return a population of new chromosomes a given size.
        
        Args:
            size: the number of individuals to generate
        """
        if not isinstance(size, (int, float)) or size < 0:
            raise ValueError('size must be a positive number')
        return [self.next_individual for i in range(0, size)]

In [365]:
arb_factory = ChromosomeFactory(BinaryChromosome, 3, 'zeros', evaluate_arb)
arb_factory.next_individual

[0 0 0]

In [368]:
class Population:
    """A class for generating, selecting from, and updating a population."""
    
    def __init__(self, size: int, factory: ChromosomeFactory):
        """
        Create a new population.
        
        Args:
            size: the size of the population
            factory: the ChromosomeFactory to use for generating individuals
        """
        # set the internal list of individuals from the factory
        self.individuals = factory.population(size)
    

In [372]:
factory = ChromosomeFactory(BinaryChromosome, 3, 'random', evaluate_arb)
pop = Population(10, factory)
pop.individuals

[[0 1 0],
 [0 0 0],
 [0 0 0],
 [1 0 0],
 [1 1 0],
 [1 1 1],
 [1 0 1],
 [0 1 1],
 [1 1 0],
 [1 0 1]]